In [60]:
from pyspark.sql.functions import col, split, concat_ws, when, size, min, trim, round, mean, lit, max, abs

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [28]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("TrackTimePrediction") \
    .getOrCreate()

spark

25/06/14 12:24:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [27]:
spark.stop()

Machine learning time

In [37]:
file_path1 = "./final_database/merged_five_thresh.csv"
df = spark.read.csv(file_path1, header=True, inferSchema=True)
df = df.filter(col("PR_seconds") >= 700)
print(df.count())
df.show(1)

3675
+--------------+---+----------+--------------------+-----+-----+------+-----+-----+-----------------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+-------+-----------------+
|  athlete_name|_c0|Unnamed: 0|      Athlete/School|Grade|State|Gender|FR_5k|SO_5k|            JR_5k|            SR_5k|FR_3200|SO_3200|JR_3200|SR_3200|FR_1600|SO_1600|JR_1600|SR_1600|FR_800|SO_800|JR_800|SR_800|     id|       PR_seconds|
+--------------+---+----------+--------------------+-----+-----+------+-----+-----+-----------------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+-------+-----------------+
|Aakersh Mathur| 16|        16|Aakersh Mathur Mo...| 2021|   CA|  Boys| NULL| NULL|961.5166666666668|952.7333333333336|   NULL|  600.4|   NULL|  559.4|   NULL|   NULL|  269.0|  260.7|  NULL|  NULL|  NULL| 123.9|8073383|919.0499992370605|
+--------------+---+----------+------------

To deal with NULLs, will try imputing with an indicator column, and simply setting the NULLs to a very large value. The first is more robust, but I worry about setting nulls to the mean since I am hoping to capture trends

In [30]:
df = df.fillna(9999999, subset=['FR_5k', 'SO_5k', 'JR_5k', 'SR_5k', 'FR_3200', 'SO_3200', 'JR_3200', 'SR_3200', 'FR_1600', 'SO_1600', 'JR_1600', 'SR_1600', 'FR_800', 'SO_800', 'JR_800', 'SR_800'])
df.show(5)

+----------------+---+----------+--------------------+-----+-----+------+---------+-----------------+-----------------+------------------+---------+---------+---------+-------+---------+---------+---------+-------+---------+---------+---------+---------+-------+------------------+
|    athlete_name|_c0|Unnamed: 0|      Athlete/School|Grade|State|Gender|    FR_5k|            SO_5k|            JR_5k|             SR_5k|  FR_3200|  SO_3200|  JR_3200|SR_3200|  FR_1600|  SO_1600|  JR_1600|SR_1600|   FR_800|   SO_800|   JR_800|   SR_800|     id|        PR_seconds|
+----------------+---+----------+--------------------+-----+-----+------+---------+-----------------+-----------------+------------------+---------+---------+---------+-------+---------+---------+---------+-------+---------+---------+---------+---------+-------+------------------+
|  Aakersh Mathur| 16|        16|Aakersh Mathur Mo...| 2021|   CA|  Boys|9999999.0|        9999999.0|961.5166666666668| 952.7333333333336|9999999.0|    60

In [31]:
feature_cols = ['FR_5k', 'SO_5k', 'JR_5k', 'SR_5k', 'FR_3200', 'SO_3200', 'JR_3200', 'SR_3200', 'FR_1600', 'SO_1600', 'JR_1600', 'SR_1600', 'FR_800', 'SO_800', 'JR_800', 'SR_800']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid='keep')
model_df = assembler.transform(df)
model_df.select('features', 'PR_seconds').show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|features                                                                                                                                                     |PR_seconds        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|[9999999.0,9999999.0,961.5166666666668,952.7333333333336,9999999.0,600.4,9999999.0,559.4,9999999.0,9999999.0,269.0,260.7,9999999.0,9999999.0,9999999.0,123.9]|919.0499992370605 |
|[9999999.0,985.8,926.9,915.0166666666668,9999999.0,597.0,574.8,567.8,9999999.0,9999999.0,267.6,264.6,9999999.0,9999999.0,9999999.0,9999999.0]                |873.5900001525879 |
|[9999999.0,9999999.0,9999999.0,956.8666666666668,9999999.0,615.7,602.8,564.7,9999999.0,9999999.0,9999999

In [32]:
train_data, test_data = model_df.randomSplit(weights=[0.8, 0.2], seed=27)
print(f"Training Dataset Count: {train_data.count()}")
print(f"Test Dataset Count: {test_data.count()}")

Training Dataset Count: 2914
Test Dataset Count: 761


In [33]:
gbt = GBTRegressor(featuresCol='features', labelCol='PR_seconds')
print("Training the GBT model...")
gbt_model = gbt.fit(train_data)
print("Training complete.")

Training the GBT model...
Training complete.


In [34]:
predictions = gbt_model.transform(test_data)
predictions.select("PR_seconds", "prediction").show(10)

+-----------------+-----------------+
|       PR_seconds|       prediction|
+-----------------+-----------------+
|917.2800006866455|929.2156087794947|
|919.0499992370605|899.3989735109267|
|909.7799997329712|911.3758098588826|
|940.8600006103516|922.3412881672614|
|953.8800010681152|937.4624097182023|
|955.7400016784668|977.1422265497764|
|876.2299995422363|874.8827581131434|
|859.1499996185303|873.0455639779069|
|898.8899993896484|880.6333919090546|
|893.9300003051758| 947.107842502953|
+-----------------+-----------------+
only showing top 10 rows


In [35]:
evaluator = RegressionEvaluator(
    labelCol="PR_seconds",
    predictionCol="prediction",
    metricName="rmse"  
)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse:.2f} seconds")
r2_evaluator = evaluator.setMetricName("r2")
r2 = r2_evaluator.evaluate(predictions)
print(f"R-squared (R2) on test data = {r2:.2f}")


Root Mean Squared Error (RMSE) on test data = 46.91 seconds
R-squared (R2) on test data = 0.27


This model sucks. It is on average 62.94 seconds off. Will try to impute with indicators next. Then add more features like progression, but realistically, I just don't have enough data yet to make a broad enough model to account for all the factors at play. Will also try using the dataset with a lower threshold. (After removing the 100s 5k times, RMSE=49.50 seconds)

Using 5 thresh, RMSE = 46.91

In [53]:
max_val = df.select(max(col("SR_800"))).first()[0]
df = df.withColumn(
    "SR_800_was_missing",
    when(col("SR_800").isNull(), 1).otherwise(0)
)
df = df.fillna(max_val, subset=["SR_800"])
df.show(1)

+--------------+---+----------+--------------------+-----+-----+------+------+------+-----------------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|  athlete_name|_c0|Unnamed: 0|      Athlete/School|Grade|State|Gender| FR_5k| SO_5k|            JR_5k|            SR_5k|FR_3200|SO_3200|JR_3200|SR_3200|FR_1600|SO_1600|JR_1600|SR_1600|FR_800|SO_800|JR_800|SR_800|     id|       PR_seconds|FR_5k_was_missing|SO_5k_was_missing|JR_5k_was_missing|SR_5k_was_missing|FR_3200_was_missing|SO_3200_was_missing|JR_3200_was_missing|SR_3200_was_missing|FR_1600_was_missing|SO_1600_was_missing|JR_1600_was_missing|S

In [54]:
output_path = './final_database'
df.coalesce(1).write \
    .option('header', 'true') \
    .mode('append') \
    .csv(output_path) 

In [55]:
feature_cols = ['FR_5k', 'SO_5k', 'JR_5k', 'SR_5k', 'FR_3200', 'SO_3200', 'JR_3200', 'SR_3200', 'FR_1600', 'SO_1600', 'JR_1600', 'SR_1600', 'FR_800', 'SO_800', 'JR_800', 'SR_800', 'FR_5k_was_missing', 'SO_5k_was_missing', 'JR_5k_was_missing', 'SR_5k_was_missing', 'FR_3200_was_missing', 'SO_3200_was_missing', 'JR_3200_was_missing', 'SR_3200_was_missing', 'FR_1600_was_missing', 'SO_1600_was_missing', 'JR_1600_was_missing', 'SR_1600_was_missing', 'FR_800_was_missing', 'SO_800_was_missing', 'JR_800_was_missing', 'SR_800_was_missing']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid='keep')
df = df.filter(col("PR_seconds") >= 700)
model_df = assembler.transform(df)
model_df.select('features', 'PR_seconds').show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|features                                                                                                                                                                                   |PR_seconds        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|[1212.0,1250.4,961.5166666666668,952.7333333333336,782.5,600.4,727.0,559.4,337.2,323.9,269.0,260.7,150.3,140.4,142.0,123.9,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0]|919.0499992370605 |
|[1212.0,985.8,926.9,915.0166666666668,782.5,597.0,574.8,567.8,337.2,323.9,267.6,264.6,150.3,140.4,142.0,152.3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1

In [56]:
train_data, test_data = model_df.randomSplit(weights=[0.8, 0.2], seed=27)
print(f"Training Dataset Count: {train_data.count()}")
print(f"Test Dataset Count: {test_data.count()}")

Training Dataset Count: 2914
Test Dataset Count: 761


In [57]:
gbt = GBTRegressor(featuresCol='features', labelCol='PR_seconds')
print("Training the GBT model...")
gbt_model = gbt.fit(train_data)
print("Training complete.")

Training the GBT model...
Training complete.


In [58]:
predictions = gbt_model.transform(test_data)
predictions.select("PR_seconds", "prediction").show(20)

+------------------+-----------------+
|        PR_seconds|       prediction|
+------------------+-----------------+
| 917.2800006866455|920.4076651790483|
| 919.0499992370605|904.8468310476235|
| 909.7799997329712|917.6375076357772|
| 940.8600006103516|924.8858759982098|
| 953.8800010681152|928.8988816005892|
| 955.7400016784668|977.2593266884074|
| 876.2299995422363|876.5921103892258|
| 859.1499996185303|877.6421014457443|
| 898.8899993896484|888.4804398747314|
| 893.9300003051758|946.0214884550903|
| 944.9199981689453|942.0453454809947|
| 920.9599990844727| 919.192528016944|
| 930.4699993133545|981.7547963022289|
|  893.439998626709|  921.99904595892|
|1072.4599990844727|978.2544579064887|
| 879.2000007629395|956.3157814492938|
| 875.2299995422363|898.5304228532132|
| 885.7400016784668|927.8638224552273|
| 969.8500003814697|915.5752108894388|
| 893.6199989318848|902.7313686508827|
+------------------+-----------------+
only showing top 20 rows


In [59]:
evaluator = RegressionEvaluator(
    labelCol="PR_seconds",
    predictionCol="prediction",
    metricName="rmse"  
)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data = {rmse:.2f} seconds")
r2_evaluator = evaluator.setMetricName("r2")
r2 = r2_evaluator.evaluate(predictions)
print(f"R-squared (R2) on test data = {r2:.2f}")

Root Mean Squared Error (RMSE) on test data = 46.97 seconds
R-squared (R2) on test data = 0.26


In [62]:
predictions_with_error = predictions.withColumn(
    "error_seconds",
    abs(col("PR_seconds") - col("prediction"))
)
predictions_with_error.select("PR_seconds", "prediction", "error_seconds").show()

+------------------+-----------------+------------------+
|        PR_seconds|       prediction|     error_seconds|
+------------------+-----------------+------------------+
| 917.2800006866455|920.4076651790483|3.1276644924028005|
| 919.0499992370605|904.8468310476235|14.203168189437065|
| 909.7799997329712|917.6375076357772| 7.857507902805992|
| 940.8600006103516|924.8858759982098|15.974124612141736|
| 953.8800010681152|928.8988816005892|24.981119467526014|
| 955.7400016784668|977.2593266884074|21.519325009940644|
| 876.2299995422363|876.5921103892258|0.3621108469894807|
| 859.1499996185303|877.6421014457443|18.492101827214015|
| 898.8899993896484|888.4804398747314|10.409559514917078|
| 893.9300003051758|946.0214884550903| 52.09148814991454|
| 944.9199981689453|942.0453454809947| 2.874652687950629|
| 920.9599990844727| 919.192528016944|1.7674710675286178|
| 930.4699993133545|981.7547963022289|51.284796988874405|
|  893.439998626709|  921.99904595892| 28.55904733221098|
|1072.45999908

In [63]:
error_threshold = 60

# Filter for rows where the error is greater than your threshold
outliers_df = predictions_with_error.filter(col("error_seconds") > error_threshold) \
    .orderBy(col("error_seconds").desc()) # Sort to see the biggest errors first

print(f"Found {outliers_df.count()} outlier predictions (error > {error_threshold} seconds).")

# Display the top outliers
print("\nRows with the largest prediction errors:")
outliers_df.show(truncate=False)

Found 130 outlier predictions (error > 60 seconds).

Rows with the largest prediction errors:
+---------------+-----+----------+--------------------------------------------------+-----+-----+------+------+-----------------+-----------------+-----------------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+-------+------------------+-----------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+
|athlete_name   |_c0  |Unnamed: 0|Athlete/School                            

Imputing with the mean and marking which rows were missing was worse: 74.15 seconds off on average. Will test the other data set next and add new features soon. (I accidentally had some 800m results as 5k times, new RMSE = 49.93). 49.28 using MAX instead of mean.

five thresh RMSE 46.97